In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset
from tqdm import tqdm

import spacy
import numpy as np

import random
import math
import time

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"# Using device: {device}")

# Using device: cuda


Then set a random seed for deterministic results/reproducability.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Instantiate our English spaCy models.

In [4]:
spacy_en = spacy.load('en')

In [5]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [6]:
SRC = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

Load our data.

In [7]:
train_seq2seq_data, valid_seq2seq_data, test_seq2seq_data = TabularDataset.splits(path='../input/cn-project1', train='train_data.csv',\
                                               validation='val_data.csv',test='test_data.csv', skip_header=True, \
                                               format = 'csv', fields = [('src',SRC),('trg',TRG)])

In [8]:
train_data, valid_data, test_data = TabularDataset.splits(path='../input/cn-hred1/', train='train.csv',\
                                               validation='val.csv',test='test.csv', skip_header=True, \
                                               format = 'csv', fields = [('src',SRC),('trg',TRG)])

We'll also print out an example.

In [9]:
print(vars(train_data.examples[0]))

{'src': ['[', "'", 'say', ',', 'jim', ',', 'how', 'about', 'going', 'for', 'a', 'few', 'beers', 'after', 'dinner', '?', "'", ',', "'", 'you', 'know', 'that', 'is', 'tempting', 'but', 'is', 'really', 'not', 'good', 'for', 'our', 'fitness', '.', "'", ']'], 'trg': ['what', 'do', 'you', 'mean', '?', 'it', 'will', 'help', 'us', 'to', 'relax', '.']}


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [10]:
SRC.build_vocab(train_seq2seq_data, min_freq = 2)
TRG.build_vocab(train_seq2seq_data, min_freq = 2)

Finally, define the `device` and create our iterators.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False)

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last tutorial, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](assets/seq2seq5.png)

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Context Encoder

In [14]:
class Context_Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.rnn = nn.LSTM(input_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        outputs, (hidden, cell) = self.rnn(src) #no cell state! 
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](assets/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [15]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](assets/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, con_enc, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.con_enc = con_enc
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == con_enc.hid_dim, "Hidden dimensions of encoder and context must be equal!"
        assert con_enc.hid_dim == decoder.hid_dim, "Hidden dimensions of context and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #last hidden state of the encoder is the context
        input_context = self.encoder(src)
        
        output_context = self.con_enc(input_context)
        
        #context also used as the initial hidden state of the decoder
        hidden = output_context
        context = hidden
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this tutorial is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [17]:
INPUT_DIM = len(SRC.vocab)
HID_DIM = 512
CON_INPUT_DIM = HID_DIM 
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256

ENC_DROPOUT = 0.5
CON_DROPOUT = 0.5 
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
con = Context_Encoder(CON_INPUT_DIM, HID_DIM, CON_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, con, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [18]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(12328, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (con_enc): Context_Encoder(
    (rnn): LSTM(512, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(12328, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=12328, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 27,357,224 trainable parameters


We initiaize our optimizer.

In [20]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [21]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [22]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator),total=len(iterator)):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [23]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator),total=len(iterator)):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [25]:
model.load_state_dict(torch.load('../input/original/tut2-model (1).pt'))

<All keys matched successfully>

In [43]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|          | 0/1023 [00:00<?, ?it/s]

Epoch: 01 | Time: 7m 28s
	Train Loss: 4.262 | Train PPL:  70.967
	 Val. Loss: 5.619 |  Val. PPL: 275.610


  0%|          | 0/1023 [00:00<?, ?it/s]

Epoch: 02 | Time: 7m 25s
	Train Loss: 4.242 | Train PPL:  69.544
	 Val. Loss: 5.620 |  Val. PPL: 275.981


  0%|          | 0/1023 [00:00<?, ?it/s]

...

Epoch: 08 | Time: 7m 23s
	Train Loss: 4.159 | Train PPL:  64.031
	 Val. Loss: 5.636 |  Val. PPL: 280.204


  0%|          | 0/1023 [00:00<?, ?it/s]

Epoch: 09 | Time: 7m 22s
	Train Loss: 4.148 | Train PPL:  63.287
	 Val. Loss: 5.662 |  Val. PPL: 287.615


100%|██████████| 96/96 [00:18<00:00,  5.09it/s]

Epoch: 10 | Time: 7m 23s
	Train Loss: 4.122 | Train PPL:  61.674
	 Val. Loss: 5.630 |  Val. PPL: 278.755


Finally, we test the model on the test set using these "best" parameters.

In [27]:
#model.load_state_dict(torch.load('ft_hred.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████| 91/91 [00:18<00:00,  4.94it/s]

| Test Loss: 5.649 | Test PPL: 284.113 |


## Inference

Now we can use our trained model to generate translations.

**Note:** these translations will be poor compared to examples shown in paper as they use hidden dimension sizes of 1000 and train for 4 days! They have been cherry picked in order to show off what attention should look like on a sufficiently sized model.

Our `translate_sentence` will do the following:
- ensure our model is in evaluation mode, which it should always be for inference
- tokenize the source sentence if it has not been tokenized (is a string)
- numericalize the source sentence
- convert it to a tensor and add a batch dimension
- get the length of the source sentence and convert to a tensor
- feed the source sentence into the encoder
- create the mask for the source sentence
- create a list to hold the output sentence, initialized with an `<sos>` token
- create a tensor to hold the attention values
- while we have not hit a maximum length
  - get the input tensor, which should be either `<sos>` or the last predicted token
  - feed the input, all encoder outputs, hidden state and mask into the decoder
  - store attention values
  - get the predicted next token
  - add prediction to current output sentence prediction
  - break if the prediction was an `<eos>` token
- convert the output sentence from indexes to tokens
- return the output sentence (with the `<sos>` token removed) and the attention values over the sequence

In [28]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    
    with torch.no_grad():
        hidden = model.encoder(src_tensor)
    
    context = hidden
    
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, context)
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

Now, we'll grab some translations from our dataset and see how well our model did. Note, we're going to cherry pick examples here so it gives us something interesting to look at, but feel free to change the `example_idx` value to look at different examples.

First, we'll get a source and target from our dataset.

## **`TRAIN SET`**

In [29]:
example_idx = 15

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['[', "'", 'are', 'you', 'all', 'right', '?', "'", ',', "'", 'i', 'will', 'be', 'all', 'right', 'soon', '.', 'i', 'was', 'terrified', 'when', 'i', 'watched', 'them', 'fall', 'from', 'the', 'wire', '.', "'", ']']
trg = ['do', "n't", 'worry', '.', 'he', 'is', 'an', 'acrobat', '。']


In [30]:
translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['wow', '!', 'wow', '!', 'thanks', '!', 'wow', '!', 'that', 'sounds', 'fantastic', '!', 'excuse', 'me', '!', 'how', 'many', 'people', 'are', 'learn', 'how', 'much', 'survive', 'survive', 'themselves', 'nowadays', '.', 'survive', 'survive', 'as', 'harvard', 'survive', 'europe', 'brings', 'pilferage', 'prescriptions', 'pilferage', 'shipments', 'cambridge', 'speciality', 'prescriptions', 'cantonese', 'languages', 'wars', 'grad', 'europe', 'shipments', 'literature', 'shipments', 'cantonese']


## **`VALIDATION SET`**

In [31]:
example_idx = 14

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['[', "'", 'what', 'qualifications', 'should', 'a', 'reporter', 'have', '?', "'", ',', "'", 'as', 'a', 'reporter', ',', 'he', 'must', 'have', 'acute', 'insight', 'and', 'language', 'skills', '.', 'at', 'the', 'same', 'time', ',', 'he', 'must', 'have', 'good', 'judgment', ',', 'the', 'respect', 'for', 'his', 'job', 'and', 'tactical', 'cooperation', 'with', 'others', '.', "'", ',', '"', 'can', 'you', 'work', 'under', 'pressure', '?', 'you', 'know', ',', 'people', 'working', 'here', 'are', 'all', 'busy', 'everyday', 'since', 'we', "'re", 'daily', 'newspaper', '.', '"', ']']
trg = ['i', 'think', 'i', "'ve", 'got', 'used', 'to', 'work', 'under', 'pressure', '.', 'i', 'will', 'adjust', 'myself', 'to', 'the', 'step', 'of', 'your', 'newspaper', 'quickly', '.']


In [32]:
translation= translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['wow', '!', 'wow', '!', 'thanks', '!', 'wow', '!', 'that', 'sounds', 'fantastic', '!', 'excuse', 'me', '!', 'how', 'many', 'people', 'are', 'learn', 'how', 'much', 'survive', 'survive', 'themselves', 'nowadays', '.', 'survive', 'survive', 'as', 'harvard', 'survive', 'europe', 'brings', 'pilferage', 'prescriptions', 'pilferage', 'shipments', 'cambridge', 'speciality', 'prescriptions', 'cantonese', 'languages', 'wars', 'grad', 'europe', 'shipments', 'literature', 'shipments', 'cantonese']


## **`TEST SET`**

In [33]:
example_idx = 25

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['[', "'", 'believe', 'it', 'or', 'not', ',', 'tea', 'is', 'the', 'most', 'popular', 'beverage', 'in', 'the', 'world', 'after', 'water', '.', "'", ',', "'", 'well', ',', 'people', 'from', 'asia', 'to', 'europe', 'all', 'enjoy', 'tea', '.', "'", ',', "'", 'right', '.', 'and', 'china', 'is', 'the', 'homeland', 'of', 'tea', '.', "'", ',', '"', 'yes', ',', 'chinese', 'people', 'love', 'drinking', 'tea', 'so', 'much', '.', 'some', 'even', 'claim', 'they', 'ca', "n't", 'live', 'without', 'tea', '.', '"', ',', "'", 'do', 'you', 'know', 'there', 'are', 'several', 'catagories', 'of', 'chinese', 'tea', '?', "'", ',', "'", 'yes', ',', 'i', 'believe', 'there', 'are', 'green', 'teas', ',', 'black', 'teas', 'and', 'scented', 'teas', '.', 'any', 'others', '?', "'", ',', "'", 'well', ',', 'have', 'you', 'ever', 'heard', 'of', 'oulong', 'tea', 'and', 'compressed', 'tea', '?', "'", ',', '"', 'oh', ',', 'yeah', '.', 'oulong', 'tea', 'is', 'good', 'for', 'one', "'s", 'health', '.', 'is', "n't", 'it'

In [34]:
translation= translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['wow', '!', 'wow', '!', 'thanks', '!', 'wow', '!', 'that', 'sounds', 'fantastic', '!', 'excuse', 'me', '!', 'how', 'many', 'people', 'are', 'learn', 'how', 'much', 'survive', 'survive', 'themselves', 'nowadays', '.', 'survive', 'survive', 'as', 'harvard', 'survive', 'europe', 'brings', 'pilferage', 'prescriptions', 'pilferage', 'shipments', 'cambridge', 'speciality', 'prescriptions', 'cantonese', 'languages', 'wars', 'grad', 'europe', 'shipments', 'literature', 'shipments', 'cantonese']


## BLEU

Previously we have only cared about the loss/perplexity of the model. However there metrics that are specifically designed for measuring the quality of a translation - the most popular is *BLEU*. Without going into too much detail, BLEU looks at the overlap in the predicted and actual target sequences in terms of their n-grams. It will give us a number between 0 and 1 for each sequence, where 1 means there is perfect overlap, i.e. a perfect translation, although is usually shown between 0 and 100. BLEU was designed for multiple candidate translations per source sequence, however in this dataset we only have one candidate per source.

We define a `calculate_bleu` function which calculates the BLEU score over a provided TorchText dataset. This function creates a corpus of the actual and predicted translation for each source sentence and then calculates the BLEU score.

## **`Corpus Bleu`**

In [35]:
import nltk 
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in tqdm(data):
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return corpus_bleu(trgs, pred_trgs)

In [36]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

100%|██████████| 5782/5782 [03:21<00:00, 28.67it/s]


BLEU score = 0.03


## **`Sentence Bleu`**

In [37]:
def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    score_bleu=0
    for datum in tqdm(data):
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        score_bleu=score_bleu+sentence_bleu([trg],pred_trg)
        
    return score_bleu/len(data)

In [38]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

  0%|          | 0/5782 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
  1%|          | 39/5782 [00:01<03:17, 29.06it/s]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
  1%|▏         | 86/5782 [00:02<03:12, 29.66it/s]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
100%|██████████| 5782/5782 [03:25<00:00, 28.11it/s]

BLEU score = 35.14


## **`Sentence Bleu with Smoothing`**

In [39]:
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    cc = SmoothingFunction()
    score_bleu=0
    for datum in tqdm(data):
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        score_bleu=score_bleu+sentence_bleu([trg],pred_trg,smoothing_function=cc.method4)
        
    return score_bleu/len(data)

In [40]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

100%|██████████| 5782/5782 [03:27<00:00, 27.85it/s]

BLEU score = 14.81


## **`DIALOG RESPONSE OUTPUT`**

In [ ]:
f = open("train_output.txt", "w",encoding='utf-8')

for example_idx in tqdm(range(len(train_data.examples))):

    src = vars(train_data.examples[example_idx])['src']
    trg = vars(train_data.examples[example_idx])['trg']

    translation = translate_sentence(src, SRC, TRG, model, device)

    print(f'-----EXAMPLE {example_idx}------', file=f)
    print('CONTEXT = ', end=' ', file=f)
    
    for i in src:
        print(i, end=' ', file=f)
        
    print(file=f)
    
    print(f'GROUND TRUTH RESPONSE = ', end=' ', file=f)

    for i in trg:
        print(i, end=' ', file=f)
        
    print(file=f)
    
    print(f'PREDICTED RESPONSE = ',end=' ', file=f)
    
    for i in translation:
        print(i, end=' ', file=f)
    
    print(file=f)
    print(file=f)

f.close()